# Building_LLM_From_Scratch__Experiment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


#### Imported Libraries and Packages

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import string
import random
import pickle

#### Runtime Type

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


#### Read the Text Corpus Demo

In [ ]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### Encoder and Decoder Function Demo

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# print(encode('hello')) # Output [61, 58, 65, 65, 68]
# print(decode([61, 58, 65, 65, 68])) # Output hello
data = torch.tensor(encode(text), dtype=torch.long)

#### Batching Demo

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 61, 62, 67, 64,  1, 68, 59],
        [71, 54, 56, 73, 62, 56, 54, 65],
        [ 1, 78, 68, 74,  1, 73, 68,  1],
        [58, 65, 65,  1, 78, 68, 74,  1]])
targets:
tensor([[61, 62, 67, 64,  1, 68, 59,  1],
        [54, 56, 73, 62, 56, 54, 65, 65],
        [78, 68, 74,  1, 73, 68,  1, 78],
        [65, 65,  1, 78, 68, 74,  1, 57]])


## Building and Training the GPT Model

#### Opening the Train or Validation Split

In [5]:
train_split_file_name = "wizard_of_oz_train_split.txt"
validation_split_file_name = "wizard_of_oz_val_split.txt"

##### Training Split Vocab Size [232 KB]

In [8]:
train_chars = ""
with open(f"{train_split_file_name}", 'r', encoding='utf-8') as f:
        text = f.read()
        train_chars = sorted(list(set(text)))

vocab_size = len(train_chars)
vocab_size
# print(train_chars)

80

##### Validation Split Vocab Size [81 KB]

In [9]:
val_chars = ""
with open(f"{validation_split_file_name}", 'r', encoding='utf-8') as f:
        text = f.read()
        val_chars = sorted(list(set(text)))

vocab_size_val = len(val_chars)
vocab_size_val
# print(vocab_size_val)

65

#### Hyperparameter Setup

In [18]:
batch_size = 32
block_size = 128
max_iters = 10000
learning_rate = 1e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

#### Encoder and Decoder Setup

In [7]:
# Add all unique characters, including special characters and numbers plus new Line characters
all_chars = string.ascii_letters + string.digits + string.punctuation + ' ' + ' \n'

# Assign the New Vocab size based on the Number of unique characters in our corpus
vocab_size = len(all_chars)

# Create string_to_int and int_to_string dictionaries
string_to_int = {ch: i for i, ch in enumerate(all_chars)}
int_to_string = {i: ch for i, ch in enumerate(all_chars)}

# Lambda functions for encoding and decoding
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Usage:
text = "Hello, Brook age 27!\nNew line."
encoded_text = encode(text)
decoded_text = decode(encoded_text)

# Output
print(f"Vocab size: {vocab_size}")
print(f"Original text: {text}")
print(f"Encoded text: {encoded_text}")
print(f"Decoded text: {decoded_text}")

Vocab size: 97
Original text: Hello, Brook age 27!
New line.
Encoded text: [33, 4, 11, 11, 14, 73, 95, 27, 17, 14, 14, 10, 95, 0, 6, 4, 95, 54, 59, 62, 96, 39, 4, 22, 95, 11, 8, 13, 4, 75]
Decoded text: Hello, Brook age 27!
New line.


#### Batching Setup with Memory Map Seeking

In [8]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = train_split_file_name if split == 'train' else validation_split_file_name
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)

            # Ensure the start position is within the valid range
            max_start_pos = file_size - block_size * batch_size

            start_pos = random.randint(0, max_start_pos)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*(batch_size-1))

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data


def get_batch(split):
    data = get_random_chunk(split)
    max_ix = len(data) - block_size - 1
    # Ensure the randomly selected indices are within the valid range
    ix = torch.randint(max_ix, (batch_size,))
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # Adjust the selected indices to prevent index out of range
    ix = torch.clamp(ix, 0, max_ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

#### Loss Estimation for Training and Evaluation

In [9]:
class EstimateLoss:
    def __init__(self, model, eval_iters, get_batch):
        self.model = model
        self.eval_iters = eval_iters
        self.get_batch = get_batch

    @torch.no_grad()
    def estimate_loss(self):
        out = {}
        self.model.eval()

        for split in ['train', 'val']:
            losses = torch.zeros(self.eval_iters)
            for k in range(self.eval_iters):
                X, Y = self.get_batch(split)
                logits, loss = self.model(X, Y)
                losses[k] = loss.item()

            out[split] = losses.mean()

        self.model.train()
        return out

#### GPT Architecture

#### Head , MultiHeadAttention , FeedForward and Block Class

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

#### GPT Language Model Class

In [11]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)


        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape


        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)
model

GPTLanguageModel(
  (token_embedding_table): Embedding(97, 384)
  (position_embedding_table): Embedding(128, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=384, out_features=96, bias=False)
            (query): Linear(in_features=384, out_features=96, bias=False)
            (value): Linear(in_features=384, out_features=96, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=

#### Training and Saving the Model

In [19]:
from tqdm import tqdm

# Create an instance of EstimateLoss
estimator = EstimateLoss(model, eval_iters, get_batch)

# Initialize the Optimizer Adam with Weight Decay
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):
    if iter % eval_iters == 0:
        # Print the input sequence before encoding
        xb, yb = get_batch('train')
        print(f"Input sequence: {decode(xb[0].tolist())}")

        losses = estimator.estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)

print('model saved')

  0%|          | 0/10000 [00:00<?, ?it/s]

Input sequence: ck signals made with
their wooden fingers or lips. Neither was there any sound to be heard
anywhere throughout the wooden countr


  0%|          | 3/10000 [00:04<3:26:06,  1.24s/it] 

step: 0, train loss: 2.320, val loss: 2.378


  1%|          | 99/10000 [00:10<10:29, 15.73it/s]

...

Input sequence: uired the Scarecrow.

"As many times as is necessary," was the reply. "I will ask the Tin
Woodman to defend the prisoner, becaus


 99%|█████████▉| 9903/10000 [19:58<00:57,  1.68it/s]

step: 9900, train loss: 0.993, val loss: 1.488


100%|██████████| 10000/10000 [20:05<00:00,  8.30it/s]


model saved


#### Chat with the New GPT

#### The following snippet is from the Validation split

You can see that the generated out put is different from the one used in the training.

**Original Text :**  
"Thank you, dear Wiz," said the grateful Scarecrow. "Now we must get the oil-can and rescue the Tin Woodman."

In [49]:
prompt = '"Thank you, dear Wiz,"'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

"Thank you, dear Wiz," remarked Dorothy, so he best held did to wander
with her people glossly were quite must be from the


#### Something unseen before creates some hallucination.   

But it attempts to form a sentence.

In [47]:
prompt = 'Hello I am Brook '
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello I am Brook as earth, as while
you says I have here to our way willy try to eat them, he answer in a
General chi


In [57]:
prompt = 'Brook '
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Brook and at
With this treat, was flumberly caught she sawho Jim
halves. Some Eureka, or course friends of
